In [17]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import re
from datetime import datetime
import numpy as np
import unidecode
import uuid
import urllib.request
import requests


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# get all Aldi link

In [2]:
url_pattern = r'(?P<group_1>[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*))'

def get_all_link_aldi():

    all_link = set()
    
    page_aldi_offers = urllib.request.urlopen("https://www.aldi.hu/hu/ajanlatok.html")
    soup = bs(page_aldi_offers)
    names = soup.body.findAll('div', {'data-hide-box' : 'false', 'class' : 'item'})

    for name in names:
        for a in name.find_all('a', href=True):
            url = a['href']
            if (url.find('hirlevel') == -1) and (url.find('online-szorolap') == -1):
                if url.startswith('https'):
                    all_link.add(url)
                else:
                    all_link.add('https://www.aldi.hu' + url)
    
    return list(all_link)

In [3]:
all_link = get_all_link_aldi()

In [11]:
all_link

['https://www.aldi.hu/hu/ajanlatok/szuper-ajanlatok-mindennap.html',
 'https://www.aldi.hu/hu/ajanlatok/szuper-ajanlatok-mindennap.html#piacterkw32',
 'https://www.aldi.hu/hu/ajanlatok/szuper-ajanlatok-mindennap.html#hetvegi',
 'https://www.aldi.hu/hu/ajanlatok/d.14-08-2022.html',
 'https://www.aldi.hu/hu/ajanlatok/d.18-08-2022.html',
 'https://www.aldi.hu/hu/ajanlatok/d.11-08-2022.html']

In [12]:
def image_download(img_url):
    print('invoked..')
    uuid_image = uuid.uuid1()
    res = requests.get(img_url)

    if res.headers['Content-Type'][res.headers['Content-Type'].find('/')+1:] == 'jpeg':

        image_name = str(uuid_image) + '.jpg'

    if res.headers['Content-Type'][res.headers['Content-Type'].find('/')+1:] == 'plain':

        image_name = str(uuid_image) + '.png'

    img_path = '/Users/kuligabor/Documents/HotDealsHungary/img/' + image_name
    urllib.request.urlretrieve(img_url , img_path)
    
    return image_name

In [13]:
def get_all_offer_aldi(all_link):
    
    def find_nth_occurrence(string, char, occurrence):
        
        val = -1
        for i in range(0, occurrence):
            val = string.find(char, val + 1)
        return val
    
    all_items = []
    
    counter = 1
    
    for url in all_link:
        
        print(f'crawl url: {url} done {counter} from {len(all_link)}')
        counter = counter + 1

        page = urllib.request.urlopen(url)
        soup = bs(page)

        if (url.find('d.') > -1) or (url.find('date.') > -1):
            print(f'{url} is date url')

            divs = soup.body.findAll('div', {'class' : ['col-md-9']})

            for div in divs:
                articles = div.findAll('article', {'class' : 'wrapper'})

                for article in articles:

                    item_dict = {}

                    div_prod = article.find('div', {'class' : 'item'})

                    item_dict['itemId'] = div_prod['data-productid'].replace('\n', '')

                    item_dict['itemName'] = article.find('h2').get_text().replace('\n', '')
                    item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()
                    
                                        
                    images = article.findAll('img')
                    img_url = np.nan
                    for img in images:
                        img_url = img['data-src']
                        break
                    
                    #item_dict['imageUrl'] = img_url
                    item_dict['imageUrl'] = image_download(img_url)

                    item_dict['price'] = article.find('span', {'class' : 'price'}).get_text().strip().replace(' Ft', '').replace(' ','')

                    measure = article.find('span', {'class' : 'additional-product-info'}).get_text().strip()
                    item_dict['measure'] = measure[:measure.find('\n')].replace('/','')

                    sales_date = article.find('p', {'class' : 'availableSoon'})

                    if sales_date != None:

                        date_pattern = r'(?P<group_1>[\d]{4}.[\d]{2}.[\d]{2})'
                        item_dict['salesStart'] = re.search(date_pattern, sales_date.get_text())[0]    

                    else:
                        item_dict['salesStart'] = np.nan

                    item_dict['source'] = url[find_nth_occurrence(url, '/', 5) + 1:url.find('.html')]
                    item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
                    item_dict['shopName'] = 'aldi'

                    if len(item_dict) > 0:
                        all_items.append(item_dict)
                        
        else:
            print(f'{url} is NOT date url')

            items = soup.body.findAll('div', {'data-hide-box' : 'false', 'class' : 'item'})

            for item in items:

                item_dict = {}

                h3 = item.find('h3')
                h2 = item.find('h2')
                

                if (h3 != None) and (h2 != None):
                    
                    try:
                        item_id = item.find('sup').get_text().replace(' ','')
                        if '/' in item_id:
                            item_id = item_id[:item_id.find('/')]
                            
                    except Exception as e:
                        
                            item_id = np.nan
                    
                    #print(h3.contents[0])
                    images = item.findAll('img')
                    img_url = np.nan
                    for img in images:
                        img_url = img['data-src']
                        break
                        
                    print(img_url)
                        
                    item_dict['itemId'] = item_id
                    item_dict['itemName'] = h3.contents[0]
                    item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()
                    #item_dict['imageUrl'] = img_url
                    item_dict['imageUrl'] = image_download(img_url)
                    
                    price = h2.find('b')

                    if price != None:
                        price_str_string = price.contents[0]
                        quantity = price_str_string[:price_str_string.find('/')].replace('Ft' , '').replace(' ','').strip()
                        measure = price_str_string[price_str_string.find('/'):].replace('/','').strip()
                        item_dict['price'] = quantity
                        item_dict['measure'] = measure

                    item_dict['salesStart'] = np.nan

                    item_dict['source'] = url[find_nth_occurrence(url, '/', 5) + 1:url.find('.html')]
                    item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
                    item_dict['shopName'] = 'aldi'
                    item_dict['isSales'] = 1

                    if len(item_dict) > 0:
                        all_items.append(item_dict)

    df = pd.DataFrame(all_items)
    
    return df

In [18]:
df = get_all_offer_aldi(all_link)

crawl url: https://www.aldi.hu/hu/ajanlatok/szuper-ajanlatok-mindennap.html done 1 from 6
https://www.aldi.hu/hu/ajanlatok/szuper-ajanlatok-mindennap.html is NOT date url
https://s7g10.scene7.com/is/image/aldi/KW32_04_hetvegi_akciomania_03
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_04_hetvegi_akciomania_05
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_04_hetvegi_akciomania_04
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_04_hetvegi_akciomania_01
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_04_hetvegi_akciomania_02
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_04_hetvegi_akciomania_06
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_01_egesz_heti_04
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_01_egesz_heti_13
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_01_egesz_heti_01
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_01_egesz_heti_15
invoked..
https://s7g10.scene7.com/is/image/aldi/KW32_01_egesz_heti_02
invoked..


KeyboardInterrupt: 

In [7]:
df

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales
0,814836,"GRANDESSA Prémium pálcikás jégkrém, 720 ml/doboz","grandessa premium palcikas jegkrem, 720 ml/doboz",https://s7g10.scene7.com/is/image/aldi/KW29_01...,1049,doboz,NaN,szuper-ajanlatok-mindennap,2022.07.21-08:01:23,aldi,1.0
1,79519,"STAROPRAMEN Világos sör vagy barna sör, 0,5 l/...","staropramen vilagos sor vagy barna sor, 0,5 l/...",https://s7g10.scene7.com/is/image/aldi/KW29_01...,259,doboz,NaN,szuper-ajanlatok-mindennap,2022.07.21-08:01:23,aldi,1.0
2,716712,"NESTEA Tea üdítőital, 1,5 l/palack","nestea tea uditoital, 1,5 l/palack",https://s7g10.scene7.com/is/image/aldi/KW29_01...,369,palack,NaN,szuper-ajanlatok-mindennap,2022.07.21-08:01:23,aldi,1.0
3,715723,"Sütni való kolbász, 420 g/csomag","sutni valo kolbasz, 420 g/csomag",https://s7g10.scene7.com/is/image/aldi/KW29_01...,699,csomag,NaN,szuper-ajanlatok-mindennap,2022.07.21-08:01:23,aldi,1.0
4,66644,"KOKÁRDÁS Vödrös tejföl, 850 g/vödör","kokardas vodros tejfol, 850 g/vodor",https://s7g10.scene7.com/is/image/aldi/KW29_01...,555,vödör,NaN,szuper-ajanlatok-mindennap,2022.07.21-08:01:23,aldi,1.0
...,...,...,...,...,...,...,...,...,...,...,...
324,000000000000323009,"PICK Friss sertés darálthús, 600g/tálca","pick friss sertes daralthus, 600g/talca",https://s7g10.scene7.com/is/image/aldi/2022070...,1399,darab,NaN,d.21-07-2022,2022.07.21-08:01:25,aldi,NaN
325,000000000000173304,Rózsa,rozsa,https://s7g10.scene7.com/is/image/aldi/2022061...,2599,darab,NaN,d.21-07-2022,2022.07.21-08:01:25,aldi,NaN
326,000000000000321959,"TENTO Toalettpapír Balsam Coconut, 8 tekercs","tento toalettpapir balsam coconut, 8 tekercs",https://s7g10.scene7.com/is/image/aldi/2022071...,899,dara,NaN,d.21-07-2022,2022.07.21-08:01:25,aldi,NaN
327,000000000000199590,Tengerifű kosár,tengerifu kosar,https://s7g10.scene7.com/is/image/aldi/2022070...,3499,dara,NaN,d.21-07-2022,2022.07.21-08:01:25,aldi,NaN


In [8]:
df = df.fillna('N.a')

In [9]:
df = df.loc[df['price'] != '']

In [10]:
df.loc[~df['price'].str.isnumeric(),
       'price'] = df.loc[~df['price'].str.isnumeric(),
       'price'].str[0:df.loc[~df['price'].str.isnumeric(),'price'].str.find(',').values[0]].astype(int)

In [11]:
df['price'] = df['price'].astype(int)

In [12]:
df[df['imageUrl'].isnull()]

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales


In [13]:
df.to_excel('aldi_result.xlsx', index=False)

# Back up

In [236]:
all_items = []

for div in divs:
    articles = div.findAll('article', {'class' : 'wrapper'})
    
    for article in articles:

        item_dict = {}
        
        div_prod = article.find('div', {'class' : 'item'})

        item_dict['item_id'] = div_prod['data-productid'].replace('\n', '')
        
        item_dict['item_name'] = article.find('h2').get_text().replace('\n', '')

        item_dict['price'] = article.find('span', {'class' : 'price'}).get_text().strip().replace(' Ft', '').replace(' ','')
        
        measure = article.find('span', {'class' : 'additional-product-info'}).get_text().strip()
        item_dict['measure'] = measure[:measure.find('\n')].replace('/','')
       
        sales_date = article.find('p', {'class' : 'availableSoon'})
        
        date_pattern = r'(?P<group_1>[\d]{4}.[\d]{2}.[\d]{2})'

        item_dict['sales_start'] = re.search(date_pattern, sales_date.get_text())[0]
        item_dict['source'] = url[find_nth_occurrence(url, '/', 5) + 1:url.find('.html')]
                
        if len(item_dict) > 0:
            all_items.append(item_dict)


AttributeError: 'NoneType' object has no attribute 'get_text'

In [232]:
all_items

[{'item_id': '000000000000307472',
  'item_name': 'ALIVE Gyermek nyári kalap vagy sapka',
  'price': '1299',
  'measure': 'darab',
  'sales_start': '2022.06.26',
  'source': 'd.26-06-2022'},
 {'item_id': '000000000321572001',
  'item_name': 'ALKA Elephant chipstallér 80g, szezámmagos',
  'price': '259',
  'measure': 'darab',
  'sales_start': '2022.06.26',
  'source': 'd.26-06-2022'},
 {'item_id': '000000000321572002',
  'item_name': 'ALKA Elephant chipstallér 80g, sós',
  'price': '259',
  'measure': 'darab',
  'sales_start': '2022.06.26',
  'source': 'd.26-06-2022'},
 {'item_id': '000000000000323767',
  'item_name': 'ARIEL Mosókapszula, 40 darab',
  'price': '3599',
  'measure': 'dara',
  'sales_start': '2022.06.26',
  'source': 'd.26-06-2022'},
 {'item_id': '000000000000323766',
  'item_name': 'ARIEL Mosókapszula, 40 darab',
  'price': '3599',
  'measure': 'dara',
  'sales_start': '2022.06.26',
  'source': 'd.26-06-2022'},
 {'item_id': '000000000155940002',
  'item_name': 'ASIA Földi

In [225]:
df = pd.DataFrame(all_items)

In [226]:
df

,item_id,item_name,price,measure,sales_start,source
0,000000000000307472,ALIVE Gyermek nyári kalap vagy sapka,1 299 Ft,darab,2022.06.26,d.26-06-2022
1,000000000321572001,"ALKA Elephant chipstallér 80g, szezámmagos",259 Ft,darab,2022.06.26,d.26-06-2022
2,000000000321572002,"ALKA Elephant chipstallér 80g, sós",259 Ft,darab,2022.06.26,d.26-06-2022
3,000000000000323767,"ARIEL Mosókapszula, 40 darab",3 599 Ft,dara,2022.06.26,d.26-06-2022
4,000000000000323766,"ARIEL Mosókapszula, 40 darab",3 599 Ft,dara,2022.06.26,d.26-06-2022
5,000000000155940002,"ASIA Földimogyoróolaj, 250ml",799 Ft,üveg,2022.06.26,d.26-06-2022
6,000000000308040002,"ASIA GREEN GARDEN Kínai készétel, 375 g, Chop-...",799 Ft,darab,2022.06.26,d.26-06-2022
7,000000000308040001,"ASIA GREEN GARDEN Kínai készétel, 375 g, Curry...",799 Ft,darab,2022.06.26,d.26-06-2022
8,000000000308040003,"ASIA GREEN GARDEN Kínai készétel, 375 g, Édes-...",799 Ft,darab,2022.06.26,d.26-06-2022
9,000000000294878003,"ASIA GREEN GARDEN Thai szósz, 300 ml, tavaszi ...",499 Ft,üveg,2022.06.26,d.26-06-2022


In [220]:
x = '/darab\n                                \n                                    (1 kg =  3,24 Ft)'

In [ ]:
item_id_pattern = r'<sup>(?P<group_1>[\d]+)<\/sup>'
match = re.search(item.get_text(), item_id_pattern)
print(match[0])

In [120]:
df

,item_name,quantity,measure,source
0,"BBQ Friss marinált csirkemellfilé, 500 g/tálca",1 299,tálca,szuper-ajanlatok-mindennap
1,"Dán szalámi, 100 g/csomag",299,csomag,szuper-ajanlatok-mindennap
2,"MILFINA Füstölt sajtszeletek, 175 g/csomag",399,csomag,szuper-ajanlatok-mindennap
3,"NATUR AKTIV Bio natúr szójaital, 1 l/doboz",399,doboz,szuper-ajanlatok-mindennap
4,"Sweetelle koktélparadicsom, 250 g/doboz",499,doboz,szuper-ajanlatok-mindennap
...,...,...,...,...
125,"Újburgonya, /kg",369,kg,piacter-ajanlatunk-a-hetre
126,"Őszibarack, 500 g/csomag",499,csomag,piacter-ajanlatunk-a-hetre
127,"Avokádó, /darab",299,darab,piacter-ajanlatunk-a-hetre
128,"Kígyóuborka, /darab",169,darab,piacter-ajanlatunk-a-hetre


In [234]:
page = urllib.request.urlopen('https://www.aldi.hu/hu/ajanlatok/d.19-06-2022.html')
soup = bs(page)

In [235]:
divs = soup.body.findAll('div', {'class' : ['col-md-9']})

In [222]:
x[:x.find('\n')]

'/darab'

In [180]:
x = 'Értékesítés kezdete 2022.06.26-tól'

In [181]:
date_pattern = r'(?P<group_1>[\d]{4}.[\d]{2}.[\d]{2})'

In [182]:
re.search(date_pattern, x)

<re.Match object; span=(20, 30), match='2022.06.26'>

In [ ]:
class="col-md-9 col-sm-12"

In [155]:
#item plp_product at-allproductteaserbox_grpl

In [156]:
articles = soup.body.findAll('article', {'class' : 'wrapper'})

In [157]:
articles

[<article class="wrapper basket_wrapper">
 <div class="item basket_product">
                             {{cartEntry.pdpUrl|getProductPathOnBasket(cartEntry.assetUrl,cartEntry.name,cartEntry.ancestorDisplayName)}}
                             {% if cartEntry.relatedServices.length %}
                             <span class="basket_product_total_price at-basket-price_lbl">{{cartEntry.totalWithServices}}</span>
                             {% else %}
                             <span class="basket_product_total_price at-basket-price_lbl">{{cartEntry.price}}</span>
                             {% endif %}
                             <div class="basket_product_price at-basket-product-unit-price_lbl">
 <strong>{{cartEntry.unitPrice}}</strong>
                                 {% if cartEntry.wasPrice %}
                                 <span class="basket_product_old_price">
                                     Eredeti ár:
                                     <s>{{cartEntry.wasPrice}}</s

In [150]:
items = soup.body.findAll('div', {'class' : ['item', 'plp_product', 'at-allproductteaserbox_grpl']})

In [151]:
items

[<div class="item basket_product">
                             {{cartEntry.pdpUrl|getProductPathOnBasket(cartEntry.assetUrl,cartEntry.name,cartEntry.ancestorDisplayName)}}
                             {% if cartEntry.relatedServices.length %}
                             <span class="basket_product_total_price at-basket-price_lbl">{{cartEntry.totalWithServices}}</span>
                             {% else %}
                             <span class="basket_product_total_price at-basket-price_lbl">{{cartEntry.price}}</span>
                             {% endif %}
                             <div class="basket_product_price at-basket-product-unit-price_lbl">
 <strong>{{cartEntry.unitPrice}}</strong>
                                 {% if cartEntry.wasPrice %}
                                 <span class="basket_product_old_price">
                                     Eredeti ár:
                                     <s>{{cartEntry.wasPrice}}</s>
 </span>
                               